# Data Analysis

Now that we've got clean data, let's start with some basic financial analysis.

First, let's load our CSV file into a DataFrame, covert our dates, set the index, and check for duplicated rows or missing values.

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/ImperialCollegeLondon/efds-ta-python/refs/heads/main/data/AAPL_2024_clean.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").sort_index().drop_duplicates()

## Returns

Returns refer to the gain or loss made on an initial investment, often expressed as a percentage. We can use the generic **percentage change** formula here:

$$ (price_{end} - price_{start}) / price_{start} $$

We can apply this to close prices to calculate the simple daily return:

$$ (close price_{today} - close price_{yesterday}) / close price_{yesterday} $$

When available, use the Adjusted Close price, which takes into account corporate actions (dividends for example).

In [ ]:
jan31_closing = df.loc["2024-01-31", "Adj Close"]
jan30_closing = df.loc["2024-01-30", "Adj Close"]

jan31_return = (jan31_closing - jan30_closing) / jan30_closing
print(f"Return on 31 Jan was {jan31_return:.2%}")

This simple daily return expresses a loss in value of 1.94% from one day to the next. Notice we leave our return in decimal form, but when we output it we use `f-strings` and `:.2%` to display it as a percentage.

If we wanted to use the above approach to calculate daily returns for each day in our data set, it would take a long time. Let's see how we can use pandas `pct_change()` to make this sort of work easy, by applying our percentage change formula one column at a time.

In [ ]:
# Create a new column and populate it with daily returns
df['Daily Return'] = df['Adj Close'].pct_change()
df

Let's now calculate cumulative returns for the period. Instead of comparing a given day with the day before it, cumulative returns compare a given day with the first day of the period, to indicate how our stock has performed since our initial investment.

We generally fill missing daily returns with a 0, which indicates no change with the day before.

Because we're doing cumulative multiplication, we'll add 1 to the adjusted closing price, so we can compound the return over time. It is generally a good idea to subtract the 1 afterwards, so you clearly see the expected return.

In [ ]:
df["Cumulative Return"] = (1 + df["Daily Return"]).cumprod() - 1 

df

### Exercise: Buy & Hold

#### Part 1

Imagine you had bought AAPL at the start of 2024. What would your expected return have been had you sold at the end of April, compared with holding until the end of May and selling then?

#### Part 2

What if you had bought at the start of May, and sold at the end of the month?

## Moving Averages

Moving averages are a different kind of indicator, one that smooths out small variations in trading data to give a better picture of the overall trend.

A Simple Moving Average (SMA) is one which averages out a price over a specific period. The average is "moving" because when a new day is considered in the period, the oldest date is discarded.

Moving averages can be *fast*, when they cover a short period, or *slow* when they consider a longer period. The longer the period, the more those small variations are smoothed out.

In [ ]:
# Calculate a fast, 20-Day Moving Average
df['20-day MA'] = df['Adj Close'].rolling(window=20).mean()

# Calculate a slow, 200-Day Moving Average
df['200-day MA'] = df['Adj Close'].rolling(window=200).mean()

## Surges

Surges in price or trading volume can be helpful indicators for traders. We can define a surge as an increase on the day before by an amount higher than some set threshold. A common threshold is two standard deviations above the mean. Since price surges consider the change in price between two trading days, we use our already calculated daily returns, which provide that value for us!

In [ ]:
# Find the mean return
mean_return = df["Daily Return"].mean()

# Define a threshold as two standard deviations above the mean
return_threshold = mean_return + (df["Daily Return"].std() * 2)

# Define a condition
condition = df["Daily Return"] > return_threshold

# Subset the dataframe where daily returns are higher than the threshold
df[condition]

### Exercise: In the news

Can you find any events associated with the days you identified as price surges?